In [ ]:
pip install selenium

In [8]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys

# 로드될 때까지 기다리는 설정을 위한 import
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

# 크롬드라이버 실행
driver = webdriver.Chrome() 

#크롬 드라이버에 url 주소 넣고 실행
driver.get('https://youtu.be/w7uwgwj6aZM?si=kMxeRqkIo6AD--pn') # 침착맨 "버거킹, 우리의 아이디어를 꼭 들어주길 바랍니다"

# 유튜브가 로드될 때까지 최대 60초 대기
search_box = WebDriverWait(driver, 60).until(
    EC.presence_of_element_located((By.NAME, "search_query"))
)